In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy Model on Vertex AI based on new model artifacts in GCS

Customer is looking to upload Tensorflow models from Cloud Storage bucket to Vertex Predictions. Customer's customer uploads model artifacts from trained models to Cloud Storage bucket. When there is a new upload, trigger [model upload and deployment](https://cloud.google.com/vertex-ai/docs/general/import-model) process to Vertex Predictions.

![Event based model deployment](./images/event_based_model_deployment.png)

High level steps

* Custom uploads trained model artifacts to Cloud Storage bucket


## Install required packages

In [ ]:
%%bash
pip install --user google-cloud-aiplatform

### Restart the kernel
Once you've installed the required packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Import the required packages

In [ ]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import time
import csv
from datetime import datetime

import matplotlib.pyplot as plt

import google.auth
from google.protobuf.json_format import MessageToDict

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google.cloud.aiplatform_v1beta1 import types
from google.cloud import bigquery
from google.cloud import exceptions

from google.cloud.aiplatform.utils import JobClientWithOverride

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow_io import bigquery as tfio_bq

- Create event trigger
- Create Cloud Run event target
  - read model artifact location in GCS
  - triggers Cloud Build to create custom container image
    - copy model artifacts to Cloud Build local from GCS
    - copy Dockerfile to Cloud Build local from GCS
    - Build image using Dockerfile and `docker build -t`
      - Copy model artifacts to the image
      - Expose custom ports
      - Start TF Serving HTTP server
  - Upload model resource with custom container image uri
  - Create endpoint
  - Deploy model to the endpoint

In [2]:
PROJECT_ID = "[your-project-id]"  # <---CHANGE THIS TO YOUR PROJECT

import os

# Get your Google Cloud project ID using google.auth
if not os.getenv("IS_TESTING"):
    import google.auth

    _, PROJECT_ID = google.auth.default()
    print("Project ID: ", PROJECT_ID)

# validate PROJECT_ID
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    print(
        f"Please set your project id before proceeding to next step. Currently it's set as {PROJECT_ID}"
    )

Project ID:  rthallam-demo-project


In [5]:
!mkdir ./tmp

mkdir: cannot create directory ‘./tmp’: File exists


## Create Event Trigger

1. Set the configuration variables used in this quickstart:

In [1]:
%%bash -s $REGION

REGION=$1

gcloud config set run/region $REGION
gcloud config set run/platform managed
gcloud config set eventarc/location $REGION

Updated property [run/region].
Updated property [run/platform].
Updated property [eventarc/location].


2. Enable Cloud Audit Logs Admin Read, Data Read, and Data Write Log Types in Google Cloud Storage:

* Read your project's IAM policy and store it in a file:

In [13]:
!gcloud projects get-iam-policy $PROJECT_ID > ./tmp/policy.yaml

* Edit your policy in `./tmp/policy.yaml`, adding or changing only the Data Access audit logs configuration for Cloud Storage.

In [15]:
%%bash

echo """
auditConfigs:
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_WRITE
  - logType: DATA_READ
  service: storage.googleapis.com""" | sed -i -e '/^auditConfigs:/{r /dev/stdin' -e 'd;}' ./tmp/policy.yaml 

* Set your new IAM policy

In [17]:
!gcloud projects set-iam-policy $PROJECT_ID ./tmp/policy.yaml

ERROR: (gcloud.projects.set-iam-policy) User [560224572293-compute@developer.gserviceaccount.com] does not have permission to access projects instance [rthallam-demo-project:setIamPolicy] (or it may not exist): Policy update access denied.


In [16]:
!cat ./tmp/policy.yaml


auditConfigs:
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_WRITE
  - logType: DATA_READ
  service: storage.googleapis.com
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_READ
  - logType: DATA_WRITE
  service: accessapproval.googleapis.com
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_READ
  - logType: DATA_WRITE
  service: dialogflow.googleapis.com
- auditLogConfigs:
  - logType: ADMIN_READ
  - logType: DATA_READ
  - logType: DATA_WRITE
  service: iam.googleapis.com
bindings:
- members:
  - serviceAccount:rt-cloudai-services@rthallam-demo-project.iam.gserviceaccount.com
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-560224572293@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-560224572293@gcp-sa-aiplatform.iam.gserviceaccount.com
  - serviceAccount:service-560224572293@gcp-sa-staging-aiplatform.iam.gserviceaccount.com
  role: roles/aipla

## Configure GCP settings

*Before running the notebook make sure to follow the repo's README file to install the pre-requisites.*

In [ ]:
from datetime import datetime
import logging
import os

# from pathlib import Path
# import subprocess
import google.auth
from google.cloud.devtools import cloudbuild_v1

logging.getLogger().setLevel(logging.INFO)
credentials, project_id = google.auth.default()
client = cloudbuild_v1.services.cloud_build.CloudBuildClient()

build_version = datetime.now().strftime("%Y%m%d%H%M%S")

dockerfile_path = os.path.join(cifar_config.path, 'Dockerfile')
gs_dockerfile_path = dockerfile_path.replace("/gcs/", "gs://")
config_prop_path = os.path.join(cifar_config.path, 'config.properties')
gs_config_prop_path = config_prop_path.replace("/gcs/", "gs://")

export_path = f"{cifar_mar.path}/model-store"
model_path = os.path.join(export_path, mar_model_name)
gs_model_path = model_path.replace("/gcs/", "gs://")
logging.warning("gs_model_path: %s", gs_model_path)

image_uri = f"gcr.io/{project}/pytorch_train_cifar10:{build_version}"
logging.info("image uri: %s", image_uri)

build = cloudbuild_v1.Build(
    images=[image_uri]
)
build.steps = [
    {
        "name": "gcr.io/cloud-builders/gsutil",
        "args": [
            "cp",
            gs_config_prop_path,
            "config.properties",
        ],
    },
    {
        "name": "gcr.io/cloud-builders/gsutil",
        "args": ["cp", f"{gs_model_path}", f"{mar_model_name}"],
    },
    {
        "name": "gcr.io/cloud-builders/gsutil",
        "args": [
            "cp",
            gs_dockerfile_path,
            "Dockerfile",
        ],
    },
    {
      "name": 'gcr.io/cloud-builders/docker',
      "args": [ 'build', '-t', image_uri, '.' ]
    }
]
operation = client.create_build(project_id=project, build=build)
print("IN PROGRESS:")
print(operation.metadata)

result = operation.result()
# Print the completed status
print("RESULT:", result.status)
return(image_uri,)

## Preparing training data in BigQuery

### Explore Chicago Taxi dataset

In [ ]:
%%bigquery data

SELECT 
    *
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
LIMIT 3

In [ ]:
data.head().T

In [ ]:
%%bigquery data

SELECT 
    CAST(EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS string) AS trip_dayofweek, 
    FORMAT_DATE('%A',cast(trip_start_timestamp as date)) AS trip_dayname,
    COUNT(*) as trip_count,
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    EXTRACT(YEAR FROM trip_start_timestamp) = 2020 
GROUP BY
    trip_dayofweek,
    trip_dayname
ORDER BY
    trip_dayofweek

In [ ]:
data

In [ ]:
data.plot(kind='bar', x='trip_dayname', y='trip_count')

### Create  data splits

In [ ]:
BQ_DATASET_NAME = f'{PREFIX}_dataset' 
BQ_TRAIN_SPLIT_NAME = 'training'
BQ_VALID_SPLIT_NAME = 'validation'
BQ_TEST_SPLIT_NAME = 'testing'
BQ_LOCATION = 'US'
SAMPLE_SIZE = 500000
YEAR = 2020

#### Create a BQ dataset to host the splits

In [ ]:
client = bigquery.Client(location=BQ_LOCATION)

dataset_id = f'{PROJECT}.{BQ_DATASET_NAME}'
dataset = bigquery.Dataset(dataset_id)

try:
    dataset = client.create_dataset(dataset, timeout=30)
    print('Created dataset: ', dataset_id)
except exceptions.Conflict:
    print('Dataset {} already exists'.format(dataset_id))

#### Create training, validation, and test splits tables

In [ ]:
sql_script_template = '''
CREATE TEMP TABLE features 
AS (
    WITH
      taxitrips AS (
      SELECT
        FORMAT_DATETIME('%Y-%d-%m', trip_start_timestamp) AS date,
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
      WHERE 1=1 
      AND pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = @YEAR
    )

    SELECT
      trip_start_timestamp,
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude), 
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      CASE (ABS(MOD(FARM_FINGERPRINT(date),10))) 
          WHEN 9 THEN 'TEST'
          WHEN 8 THEN 'VALIDATE'
          ELSE 'TRAIN' END AS data_split
    FROM
      taxitrips
    LIMIT @LIMIT
);

CREATE OR REPLACE TABLE `@PROJECT.@DATASET.@TRAIN_SPLIT`
AS
SELECT * EXCEPT (trip_start_timestamp, data_split)
FROM features
WHERE data_split='TRAIN';

CREATE OR REPLACE TABLE `@PROJECT.@DATASET.@VALIDATE_SPLIT`
AS
SELECT * EXCEPT (trip_start_timestamp, data_split)
FROM features
WHERE data_split='VALIDATE';

CREATE OR REPLACE TABLE `@PROJECT.@DATASET.@TEST_SPLIT`
AS
SELECT * EXCEPT (trip_start_timestamp, data_split)
FROM features
WHERE data_split='TEST';

DROP TABLE features;
'''

In [ ]:
sql_script = sql_script_template.replace(
    '@PROJECT', PROJECT).replace(
    '@DATASET', BQ_DATASET_NAME).replace(
    '@TRAIN_SPLIT', BQ_TRAIN_SPLIT_NAME).replace(
    '@VALIDATE_SPLIT', BQ_VALID_SPLIT_NAME).replace(
    '@TEST_SPLIT', BQ_TEST_SPLIT_NAME).replace(
    '@YEAR', str(YEAR)).replace(
    '@LIMIT', str(SAMPLE_SIZE))

job = client.query(sql_script)
job.result()    

#### Review splits

In [ ]:
sql_script = f'''
SELECT * 
FROM `{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}`
'''

data = client.query(sql_script).to_dataframe()

In [ ]:
data.head()

## Submitting Vertex training jobs

![Training pipeline](../images/custom-training-on-vertex-ai.png)

### Display the model

`tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")`

![Model](model.png)

### Prepare a training script

In [ ]:
SCRIPT_FOLDER = 'trainer'
if tf.io.gfile.exists(SCRIPT_FOLDER):
    tf.io.gfile.rmtree(SCRIPT_FOLDER)
tf.io.gfile.mkdir(SCRIPT_FOLDER)
file_path = os.path.join(SCRIPT_FOLDER, 'train.py')

In [ ]:
%%writefile {file_path}

# Copyright 2021 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and

import hypertune
import json
import os
import re
import tensorflow as tf

from absl import app
from absl import flags
from absl import logging

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow_io import bigquery as tfio_bq

from tensorboard.plugins.hparams import api as tb_hp

FLAGS = flags.FLAGS
flags.DEFINE_integer('epochs', 3, 'Nubmer of epochs')
flags.DEFINE_integer('units', 32, 'Number units in a hidden layer')
flags.DEFINE_integer('per_replica_batch_size', 128, 'Per replica batch size')
flags.DEFINE_float('dropout_ratio', 0.5, 'Dropout ratio')
flags.DEFINE_string('training_table', None, 'Training table name')
flags.DEFINE_string('validation_table', None, 'Validationa table name')
flags.mark_flag_as_required('training_table')
flags.mark_flag_as_required('validation_table')

LOCAL_MODEL_DIR = '/tmp/saved_model'
LOCAL_TB_DIR = '/tmp/logs'
LOCAL_CHECKPOINT_DIR = '/tmp/checkpoints'
EVALUATION_FILE_NAME = 'evaluations.json'

# Define features
FEATURES = {
    "tip_bin": ("categorical", tf.int64),
    "trip_month": ("categorical", tf.int64),
    "trip_day": ("categorical", tf.int64),
    "trip_day_of_week": ("categorical", tf.int64),
    "trip_hour": ("categorical", tf.int64),
    "payment_type": ("categorical", tf.string),
    "pickup_grid": ("categorical", tf.string),
    "dropoff_grid": ("categorical", tf.string),
    "euclidean": ("numeric", tf.double),
    "trip_seconds": ("numeric", tf.int64),
    "trip_miles": ("numeric", tf.double),
}
TARGET_FEATURE_NAME = 'tip_bin'

 # Set hparams for Tensorboard and Vertex hp tuner
HP_DROPOUT = tb_hp.HParam("dropout")
HP_UNITS = tb_hp.HParam("units")
HPARAMS = [
    HP_UNITS,
    HP_DROPOUT,
]
METRICS = [
    tb_hp.Metric(
        "epoch_accuracy",
        group="validation",
        display_name="epoch accuracy"),
]
HPTUNE_METRIC = 'val_accuracy'
    

def set_job_dirs():
    """Sets job directories and hyperparameter tuning trial id
    based on env variables set by Vertex AI."""
    
    model_dir = os.getenv('AIP_MODEL_DIR', LOCAL_MODEL_DIR)
    tb_dir = os.getenv('AIP_TENSORBOARD_LOG_DIR', LOCAL_TB_DIR)
    checkpoint_dir = os.getenv('AIP_CHECKPOINT_DIR', LOCAL_CHECKPOINT_DIR)
    
    path = os.path.normpath(tb_dir)
    trial_id = re.match('^[0-9]+$', path.split(os.sep)[-2])
    if not trial_id:
        trial_id = '0'
    else:
        trial_id = trial_id[0]
    logging.info(trial_id)
    
    return model_dir, tb_dir, checkpoint_dir, trial_id


def get_bq_dataset(table_name, selected_fields, target_feature='tip_bin', batch_size=32):
    
    def _transform_row(row_dict):
        trimmed_dict = {column:
                       (tf.strings.strip(tensor) if tensor.dtype == 'string' else tensor) 
                       for (column,tensor) in row_dict.items()
                       }
        target = trimmed_dict.pop(target_feature)
        return (trimmed_dict, target)

    project_id, dataset_id, table_id = table_name.split('.')
    
    client = tfio_bq.BigQueryClient()
    parent = f'projects/{project_id}'

    read_session = client.read_session(
        parent=parent,
        project_id=project_id,
        table_id=table_id,
        dataset_id=dataset_id,
        selected_fields=selected_fields,
    )

    dataset = read_session.parallel_read_rows().map(_transform_row).batch(batch_size)
    
    return dataset


def get_category_encoding_layer(name, dataset, dtype):
    """Creates a CategoryEncoding layer for a given feature."""

    if dtype == tf.string:
      index = preprocessing.StringLookup()
    else:
      index = preprocessing.IntegerLookup()

    feature_ds = dataset.map(lambda x, y: x[name])
    index.adapt(feature_ds)
    encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

    return lambda feature: encoder(index(feature))


def get_normalization_layer(name, dataset):
  """"Creates a Normalization layer for a given feature."""
  normalizer = preprocessing.Normalization()

  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)

  return normalizer


def create_model(dataset, input_features, units, dropout_ratio):
    """Creates a binary classifier for Chicago Taxi tip prediction task."""
    
    all_inputs = []
    encoded_features = []
    for feature_name, feature_info in input_features.items():
        col = tf.keras.Input(shape=(1,), name=feature_name, dtype=feature_info[1])
        if feature_info[0] == 'categorical':
            
            encoding_layer = get_category_encoding_layer(feature_name, 
                                                         dataset,
                                                         feature_info[1])
        else:
            encoding_layer = get_normalization_layer(feature_name,
                                                     dataset) 
        encoded_col = encoding_layer(col)
        all_inputs.append(col)
        encoded_features.append(encoded_col)
        
    all_features = tf.keras.layers.concatenate(encoded_features)
    
    x = tf.keras.layers.Dense(units, activation="relu")(all_features)
    x = tf.keras.layers.Dropout(dropout_ratio)(x)
    output = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(all_inputs, output)
    
    return model


class HptuneCallback(tf.keras.callbacks.Callback):
    """
    A custom Keras callback class that reports a metric to hypertuner
    at the end of each epoch.
    """
    
    def __init__(self, metric_tag, metric_value):
        super(HptuneCallback, self).__init__()
        self.metric_tag = metric_tag
        self.metric_value = metric_value
        self.hpt = hypertune.HyperTune()
        
    def on_epoch_end(self, epoch, logs=None):
        self.hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag=self.metric_tag,
            metric_value=logs[self.metric_value],
            global_step=epoch)
        

def main(argv):
    del argv
    
    # Set distribution strategy
    strategy = tf.distribute.MirroredStrategy()
    
    global_batch_size = (strategy.num_replicas_in_sync *
                         FLAGS.per_replica_batch_size)
    
    # Prepare datasets
    selected_fields = {key: {'output_type': value[1]} for key, value in FEATURES.items()}
    validation_ds = get_bq_dataset(FLAGS.validation_table, 
                                   selected_fields, 
                                   batch_size=global_batch_size)
    training_ds = get_bq_dataset(FLAGS.training_table,
                                 selected_fields,
                                 batch_size=global_batch_size)
    
    # Configure Tensorboard hparams
    model_dir, tb_dir, checkpoint_dir, trial_id = set_job_dirs()
    with tf.summary.create_file_writer(tb_dir).as_default():
        tb_hp.hparams_config(hparams=HPARAMS, metrics=METRICS)
        
    hparams = {
        HP_UNITS: FLAGS.units,
        HP_DROPOUT: FLAGS.dropout_ratio
    }
    
    # Create the model
    input_features = {key: value for key, value in FEATURES.items() if key != TARGET_FEATURE_NAME}
    logging.info('Creating the model ...')
    with strategy.scope():
        model = create_model(training_ds, input_features, hparams[HP_UNITS], hparams[HP_DROPOUT])
        model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # Configure training regimen
    callbacks = [tf.keras.callbacks.experimental.BackupAndRestore(backup_dir=checkpoint_dir)]
    callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                                    update_freq='batch',
                                                    profile_batch=0))
    callbacks.append(tb_hp.KerasCallback(writer=tb_dir, 
                                         hparams=hparams,
                                         trial_id=trial_id))
    callbacks.append(HptuneCallback(HPTUNE_METRIC, HPTUNE_METRIC))
    
    # Start training
    logging.info('Starting training ...')
    history = model.fit(training_ds, 
              epochs=FLAGS.epochs, 
              validation_data=validation_ds,
              callbacks=callbacks)
    
    # Save trained model
    logging.info('Training completed. Saving the trained model to: {}'.format(model_dir))
    model.save(model_dir)  
    
    
if __name__ == '__main__':
    logging.set_verbosity(logging.INFO)
    app.run(main)

### Initialize Vertex AI SDK and Set an Experiment

Define experiment name.

In [ ]:
EXPERIMENT_NAME = "chicago-taxi-tips-classifier"  # @param {type:"string"}

If EXPERIMENT_NAME is not set, set a default one below:

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
if EXPERIMENT_NAME == "" or EXPERIMENT_NAME is None:
    EXPERIMENT_NAME = "my-experiment-" + TIMESTAMP

In [ ]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET,
    experiment=EXPERIMENT_NAME
)

### Configure and submit a custom Vertex job using a script

In [ ]:
job_name = f"chicago-taxi-clsfr-custom-{TIMESTAMP}"
base_output_dir = f'{STAGING_BUCKET}/jobs/{job_name}'

epochs = 2
batch_size = 128

#container_uri = 'us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-4:latest'
container_uri = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-4:latest'
requirements = ['tensorflow-datasets==4.3.0']
args = [
    f'--epochs={epochs}', 
    f'--per_replica_batch_size={batch_size}',
    '--training_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
    '--validation_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
]

machine_type = 'n1-standard-4'
#accelerator_type = 'NVIDIA_TESLA_T4'
#accelerator_count = 1

job = vertex_ai.CustomJob.from_local_script(
    display_name=job_name,
    machine_type=machine_type,
#    accelerator_type=accelerator_type,
#    accelerator_count=accelerator_count,
    script_path='trainer/train.py',
    container_uri=container_uri,
    requirements=requirements,
    args=args,
    staging_bucket=base_output_dir
)


vertex_ai.start_run("custom-training-run-1")  # Change this to your desired run name
parameters = {"epochs": 2, 
              "batch_size": 128,
              "training_table": f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
              "validation_table": f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}'
             }
vertex_ai.log_params(parameters)

In [ ]:
job.run(sync=False, 
        service_account=VERTEX_SA,
        tensorboard=TENSORBOARD,
        )

In [ ]:
print(f"Job Name: {job.display_name}")
print(f"Job Resource Name: {job.resource_name}\n")
print(f"Check training progress at {job._dashboard_uri()}")

Check model artifacts in GCS bucket

In [ ]:
!gsutil ls $base_output_dir/model/

Model artifacts to be deployed to Vertex Model Resource later in the notebook

### Configure and submit a Vertex job using a custom container

![training-with-custom-containers-on-vertex-training](../images/training-with-custom-containers-on-vertex-training.png)

#### Create a Dockerfile

In [ ]:
BASE_IMAGE = 'gcr.io/deeplearning-platform-release/tf2-cpu.2-4'
TRAIN_IMAGE = f'gcr.io/{PROJECT}/{PREFIX}_chicago_taxi_clsfr_trainer'

dockerfile = f'''
FROM {BASE_IMAGE}

WORKDIR /trainer
RUN pip install cloudml-hypertune

# Copies the trainer code to the docker image.
COPY train.py .

ENTRYPOINT ["python", "train.py"]
'''

with open(os.path.join(SCRIPT_FOLDER, 'Dockerfile'), 'w') as f:
    f.write(dockerfile)

#### Build a container image

In [ ]:
!gcloud builds submit --tag {TRAIN_IMAGE} {SCRIPT_FOLDER}

#### Prepare worker pool specification

In [ ]:
epochs = 2
batch_size = 128

worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
#            "accelerator_type": "NVIDIA_TESLA_V100",
#            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_IMAGE,
#            "command": ["python", "train.py"],
            "args": [
                f'--epochs={epochs}', 
                f'--per_replica_batch_size={batch_size}',
                '--training_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
                '--validation_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_VALID_SPLIT_NAME}',
            ],
        },
    }
]

#### Submit and monitor the job

In [ ]:
job_name = f"chicago-taxi-clsfr-custom-{time.strftime('%Y%m%d_%H%M%S')}"

job = vertex_ai.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=f'{STAGING_BUCKET}/{job_name}'
)

vertex_ai.start_run("custom-training-run-2")  # Change this to your desired run name
parameters = {"epochs": epochs, 
              "batch_size": batch_size,
              "training_table": f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
              "validation_table": f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}'
             }
vertex_ai.log_params(parameters)

In [ ]:
job.run(sync=False, 
        service_account=VERTEX_SA,
        tensorboard=TENSORBOARD
)

In [ ]:
print(f"Job Name: {job.display_name}")
print(f"Job Resource Name: {job.resource_name}\n")
print(f"Check training progress at {job._dashboard_uri()}")

In [ ]:
!gsutil ls {STAGING_BUCKET}/{job_name}

### Configure and submit a Hyperparameter Tuning job

In [ ]:
epochs = 5
batch_size = 128

# The spec of the worker pools including machine type and Docker image
# Be sure to replace IMAGE_URI with the path to your Docker image in GCR
worker_pool_specs = [{
    "machine_spec": {
        "machine_type": "n1-standard-4",
      #  "accelerator_type": vertex_ai.gapic.AcceleratorType.NVIDIA_TESLA_T4,
      #  "accelerator_count": 1,
    },
    "replica_count": 1,
    "container_spec": {
        "image_uri": TRAIN_IMAGE,
        "args": [
            f'--epochs={epochs}', 
            f'--per_replica_batch_size={batch_size}',
            '--training_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_TRAIN_SPLIT_NAME}',
            '--validation_table=' + f'{PROJECT}.{BQ_DATASET_NAME}.{BQ_VALID_SPLIT_NAME}',
        ],
    },
}]

# Dicionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec={'val_accuracy':'maximize'}

# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "units": hpt.DiscreteParameterSpec(values=[32, 64], scale=None),
    "dropout_ratio": hpt.DoubleParameterSpec(min=0.4, max=0.6, scale="linear")
}

#### Configure custom job

In [ ]:
TIMESTAMP = time.strftime('%Y%m%d_%H%M%S')

In [ ]:
job_name = f"chicago-taxi-clsfr-custom-hp-{TIMESTAMP}"

job = vertex_ai.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=f'{STAGING_BUCKET}/{job_name}'
)

#### Create and run Hyperparameter Tuning job

In [ ]:
hp_job_name = f"chicago-taxi-clsfr-hptuning-{TIMESTAMP}"

hp_job = vertex_ai.HyperparameterTuningJob(
    display_name=hp_job_name,
    custom_job=job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=3,
    parallel_trial_count=3,
    max_failed_trial_count=1
)

In [ ]:
hp_job.run(sync=False, 
        service_account=VERTEX_SA,
        tensorboard=TENSORBOARD
)

#### Monitor the job

In [ ]:
print(f"Job Name: {hp_job.display_name}")
print(f"Job Resource Name: {hp_job.resource_name}\n")
print(f"Check training progress at {hp_job._dashboard_uri()}")

print(f"Job state {hp_job.state}")

#### Wait for tuning to complete

In [ ]:
while True:
    if hp_job.state != vertex_ai.gapic.JobState.JOB_STATE_SUCCEEDED:
        print("Study trials have not completed:", hp_job.state)
        if (hp_job.state == vertex_ai.gapic.JobState.JOB_STATE_FAILED or 
           hp_job.state == vertex_ai.gapic.JobState.JOB_STATE_CANCELLED):
            break
    else:
        print("Study trials have completed")
        break
    time.sleep(60)

#### Review the results of the study

In [ ]:
pd.set_option('display.max_colwidth', None)

df_hp_job_results = pd.DataFrame([MessageToDict(trial.__class__.pb(trial)) for trial in hp_job.trials])
df_hp_job_results["parameters"] = df_hp_job_results.parameters.apply(lambda x: {item["parameterId"]:item["value"] for item in x})
df_hp_job_results["metrics"] = df_hp_job_results.finalMeasurement.apply(lambda x: x["metrics"][0]["value"])
df_hp_job_results.drop("finalMeasurement", axis=1, inplace=True)
df_hp_job_results

In [ ]:
best_accuracy = 0.0
best_trial = df_hp_job_results.iloc[[df_hp_job_results["metrics"].idxmax()]]
best_trial

#### Get the best model

In [ ]:
model_dir = f'{STAGING_BUCKET}/{job_name}'

!gsutil ls {model_dir}

In [ ]:
best_model_dir = '{}/{}'.format(model_dir, best_trial.id.values[0])

!gsutil ls {best_model_dir}

## Deploying a model to Vertex 

### Inspect the model

In [ ]:
saved_model_path = f'{best_model_dir}/model'

!saved_model_cli show --dir {saved_model_path} --tag_set serve --signature_def serving_default

### Upload the model using pre-built serving container

In [ ]:
display_name = f'{PREFIX} Chicago Taxi Tip Classifier'
description = 'Chicago Taxi Tip TensorFlow classifier'
serving_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest'

model = vertex_ai.Model.upload(
    display_name=display_name,
    description=description,
    artifact_uri=saved_model_path,
    serving_container_image_uri=serving_image_uri
)

model.wait()

### Create an endpoint

In [ ]:
display_name = f'{PREFIX} Chicago Taxi Tip Classifier Endpoint'

endpoint = vertex_ai.Endpoint.create(display_name=display_name)

### Deploy the Model

In [ ]:
deployed_model_display_name = f'{PREFIX}-taxi-v1'
traffic_percentage = 100
machine_type = 'n1-standard-4'

endpoint = model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        machine_type=machine_type,
        traffic_percentage=traffic_percentage)


## Invoking the deployed model using Vertex SDK

### Get the endpoint

In [ ]:
filter = f'display_name="{PREFIX} Chicago Taxi Tip Classifier Endpoint"'

for endpoint_info in vertex_ai.Endpoint.list(filter=filter):
    print(endpoint_info)
    
endpoint = vertex_ai.Endpoint(endpoint_info.resource_name)

### Call the endpoint

In [ ]:
test_instances = [  
    
    {
        "dropoff_grid": ["POINT(-87.6 41.9)"],
        "euclidean": [2064.2696],
        "payment_type": ["Credit Card"],
        "pickup_grid": ["POINT(-87.6 41.9)"],
        "trip_miles": [1.37],
        "trip_day": [12],
        "trip_hour": [16],
        "trip_month": [2],
        "trip_day_of_week": [4],
        "trip_seconds": [555]
    }
]

predictions = endpoint.predict(instances=test_instances)
prob = tf.nn.sigmoid(predictions[0])
print('Probability of tip > 20%:', prob.numpy())

## Upload the model using custom serving container

You might want to use a custom serving container with Vertex Predictions for any of the following reasons:

- To serve predictions from an ML model trained using a framework other than TensorFlow, scikit-learn, or XGBoost
- To preprocess prediction requests or postprocess the predictions generated by your model
- To run a prediction server written in a programming language of your choice
- To install dependencies that you want to use to customize prediction

When you use a custom container, Vertex AI runs a Docker container of your choice on each prediction node. The [container image requirements](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements) must meet to be compatible with Vertex AI.

Following diagram shows the steps to deploy a model on vertex AI with custom container.

![serving with custom container on Vertex AI](../images/serving-with-custom-containers-on-vertex-predictions.png)

Let's deploy the previously trained Tensorflow/Keras model on Vertex AI with Tensorflow Serving custom container image.

### Create Dockerfile with serving container image

Dockerfile image consist of following steps

1. Create image from base Tensorflow serving image `tensorflow/serving:2.4`
2. Copy model artifacts (Tensorflow savedModel) generated from the training job (available in GCS bucket) in the image
3. Run Tensorflow Server process in the backend listening to prediction requests on port 8080

In [ ]:
SCRIPT_FOLDER = 'predictor'
if tf.io.gfile.exists(SCRIPT_FOLDER):
    tf.io.gfile.rmtree(SCRIPT_FOLDER)
tf.io.gfile.mkdir(SCRIPT_FOLDER)
dockerfile_path = os.path.join(SCRIPT_FOLDER, 'Dockerfile')

#### Download Model Artifacts from GCS Bucket

In [ ]:
saved_model_path = "gs://rtvw-rt-vertex-sandbox-bucket/chicago-taxi-clsfr-custom-hp-20210621_002500/1/model"

In [ ]:
print(f"Saved model artifacts from the training job located at {saved_model_path}")

In [ ]:
!gsutil ls gs://rtvw-rt-vertex-sandbox-bucket/chicago-taxi-clsfr-custom-hp-20210621_002500/1/model/

In [ ]:
MODEL_FOLDER = 'model'
if tf.io.gfile.exists(MODEL_FOLDER):
    tf.io.gfile.rmtree(MODEL_FOLDER)
tf.io.gfile.mkdir(f"{SCRIPT_FOLDER}/{MODEL_FOLDER}")

In [ ]:
print(f"{SCRIPT_FOLDER}/{MODEL_FOLDER}")

In [ ]:
!gsutil cp -r gs://rtvw-rt-vertex-sandbox-bucket/chicago-taxi-clsfr-custom-hp-20210621_002500/1/model/* $SCRIPT_FOLDER/$MODEL_FOLDER/

#### Create Dockerfile

In [ ]:
%%writefile {dockerfile_path}

FROM tensorflow/serving

# Set where models should be stored in the container
ENV MODEL_BASE_PATH=/models
ENV MODEL_NAME=model

# Create models dir
RUN mkdir -p ${MODEL_BASE_PATH}/${MODEL_NAME}/1

# COPY model files
COPY model ${MODEL_BASE_PATH}/${MODEL_NAME}/1/

# Create a script that runs the model server so we can use environment variables
# while also passing in arguments from the docker command line
RUN echo '#!/bin/bash \n\n\
tensorflow_model_server --port=8500 --rest_api_port=8501 \
--model_name=${MODEL_NAME} --model_base_path=${MODEL_BASE_PATH}/${MODEL_NAME} \
"$@"' > /usr/bin/predictor.sh \
&& chmod +x /usr/bin/predictor.sh

# REST API port
EXPOSE 8501

# Remove entrypoint from parent image
ENTRYPOINT []

CMD ["/usr/bin/predictor.sh"]

In [ ]:
%%writefile {dockerfile_path}

FROM tensorflow/serving:2.4.0

# Set where models should be stored in the container
ENV MODEL_BASE_PATH=/models
ENV MODEL_NAME=model

RUN mkdir -p ${MODEL_BASE_PATH}/${MODEL_NAME}/1

# copy the model file
COPY model ${MODEL_BASE_PATH}/${MODEL_NAME}/1/

# Create a script that runs the model server so we can use environment variables
# while also passing in arguments from the docker command line
RUN echo '#!/bin/bash \n\n\
tensorflow_model_server --port=5000 --rest_api_port=8080 \
--model_name=${MODEL_NAME} --model_base_path=${MODEL_BASE_PATH}/${MODEL_NAME} \
"$@"' > /usr/bin/predictor.sh \
&& chmod +x /usr/bin/predictor.sh

EXPOSE 5000
EXPOSE 8080

# Remove entrypoint from parent image
ENTRYPOINT []

CMD ["/usr/bin/predictor.sh"]

### Build serving image with Cloud Build

In [ ]:
CUSTOM_SERVING_IMAGE = f'gcr.io/{PROJECT}/{PREFIX}_chicago_taxi_clsfr_serving'

In [ ]:
{dockerfile_path}

In [ ]:
!gcloud builds submit --tag {CUSTOM_SERVING_IMAGE} {SCRIPT_FOLDER}

#### Testing predictions locally [Optional]

Run below commands on Cloud Shell locally to test predictions. You cannot run them on Vertex Notebooks since port 8080 is already in use.

In [ ]:
!docker stop local-taxi-tip-clsfr
!docker pull {CUSTOM_SERVING_IMAGE}
!docker run -t -d --rm -p 8080:8080 \
    --name=local-taxi-tip-clsfr \
    {CUSTOM_SERVING_IMAGE}
!docker container ls
!sleep 10
!curl http://0.0.0.0:8080/v1/models/model
!curl -d @instances.json -X POST http://0.0.0.0:8080/v1/models/model:predict

```
$ curl http://0.0.0.0:8080/v1/models/model
{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}
$ curl -d @instances.json -X POST http://0.0.0.0:8080/v1/models/model:predict
{
    "predictions": [[1.11188579]
    ]
}
```

### Deploy Model Resource with Custom Container

In [ ]:
display_name = f'{PREFIX} Chicago Taxi Tip Classifier Custom Serving - debug'
description = 'Chicago Taxi Tip TensorFlow classifier with TF Serving'

MODEL_BASE_PATH = "models"
MODEL_NAME = "model"
health_route = f"/v1/{MODEL_BASE_PATH}/{MODEL_NAME}"
predict_route = f"/v1/{MODEL_BASE_PATH}/{MODEL_NAME}:predict"

custom_model = vertex_ai.Model.upload(
    display_name=display_name,
    description=description,
    serving_container_image_uri=CUSTOM_SERVING_IMAGE,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route
    # serving_container_ports=[8501]
)

custom_model.wait()

In [ ]:
custom_model

### Create an Endpoint for Model with Custom Container

In [ ]:
display_name = f'{PREFIX} Chicago Taxi Tip Classifier Endpoint Custom Serving - debug'

custom_endpoint = vertex_ai.Endpoint.create(display_name=display_name)

### Deploy the Model to Endpoint

---

**NOTE: When deploying the model with custom container (with TF Serving) to Vertex Endpoints, it gets stuck. This is an open issue we are tracking.**

Error you may encounter

```
FailedPrecondition: 400 Error: model server never became ready. Please validate that your model file or container configuration are valid. Model server logs can be found at https://console.cloud.google.com/logs/viewer?...
```

---

In [ ]:
custom_model.display_name

In [ ]:
# deployed_model_display_name = f'{PREFIX}-taxi-v1-debug'
traffic_percentage = 100
machine_type = 'n1-standard-4'

custom_endpoint = custom_model.deploy(
        endpoint=custom_endpoint,
        deployed_model_display_name=custom_model.display_name,
        machine_type=machine_type,
        traffic_percentage=traffic_percentage)

## Invoking the Deployed Model using Vertex SDK

In [ ]:
filter = f'display_name="{PREFIX} Chicago Taxi Tip Classifier Endpoint Custom Serving - debug"'

for endpoint_info in vertex_ai.Endpoint.list(filter=filter):
    print(endpoint_info)
    
custom_endpoint = vertex_ai.Endpoint(endpoint_info.resource_name)

In [ ]:
custom_endpoint = vertex_ai.Endpoint('projects/437422844431/locations/us-central1/endpoints/3542837570926804992')

In [ ]:
test_instances = [  
    {
        "dropoff_grid": ["POINT(-87.6 41.9)"],
        "euclidean": [2064.2696],
        "payment_type": ["Credit Card"],
        "pickup_grid": ["POINT(-87.6 41.9)"],
        "trip_miles": [1.37],
        "trip_day": [12],
        "trip_hour": [16],
        "trip_month": [2],
        "trip_day_of_week": [4],
        "trip_seconds": [555]
    }
]

predictions = custom_endpoint.predict(instances=test_instances)
prob = tf.nn.sigmoid(predictions[0])
print('Probability of tip > 20%:', prob.numpy())

## Clean up

### Undeploy models

In [ ]:
endpoint.list_models()

In [ ]:
endpoint.undeploy_all()

### Delete endpoint

In [ ]:
endpoint.delete()

### Delete model

In [ ]:
model.delete()